In [1]:
import os
from mistralai import Mistral
from dotenv import load_dotenv
from datasets import load_from_disk
load_dotenv()

True

In [2]:
dataset_dict = load_from_disk("../../data/complete_dataset/")

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 495
    })
    validation: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 124
    })
})


In [3]:
import pandas as pd

validation_df = pd.DataFrame(dataset_dict['validation'])
df = validation_df[['answer_urdu']].copy()
df['id'] = range(len(df))
df.rename(columns={'answer_urdu': 'answers'}, inplace=True)
df = df[['id', 'answers']]

display(df)

,id,answers
0,0,اسلامی کونسل کا کوئی رکن تین سال کی مدت کے لیے...
1,1,پاکستان میں یوم آغاز سے قبل قبائلی علاقے جات م...
2,2,نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی ک...
3,3,وفاقی حکومت یا، جیسی بھی صورت ہو، صوبائی حکومت...
4,4,کسی رکن کو منحرف قرار دینے سے پہلے، پارٹی کے س...
...,...,...
119,119,وزیر اعظم مشترکہ مفادات کی کونسل کا چیئرمین ہو...
120,120,نہیں، کسی شخص کو شہادت حاصل کرنے کی غرض سے اذی...
121,121,سینیٹ میں اسپیکر اور ڈپٹی اسپیکر کے مساوی عہدے...
122,122,مالی بل کی ابتدا قومی اسمبلی میں ہوگی۔


In [4]:
api_key = os.environ["MISTRAL_API_KEY"]

def get_legal_expert_response(context, question):
    model = "mistral-large-latest"
    client = Mistral(api_key=api_key)

    chat_response = client.chat.complete(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a Pakistani legal expert who speaks in Urdu. You only respond in Urdu script.",
            },
            {
                "role": "user",
                "content": f"""Below is a context provided, followed by a question. Your task is to answer the question based on the given context. The answer should consist only of the response to the question in Urdu and should not include any other information or commentary.

    Context: ```{context}```

    Question: ```{question}```

    Answer:""",
            },
        ]
    )

    return chat_response.choices[0].message.content

In [5]:
from tqdm import tqdm

predicted_answers = []

for i in tqdm(range(len(dataset_dict['validation'])), desc="Processing responses", unit="item"):
    test_point = dataset_dict['validation'][i]
    
    context = test_point['context_urdu']
    question = test_point['question_urdu']
    
    predicted_answer = get_legal_expert_response(context=context, question=question)
    
    predicted_answers.append(predicted_answer)

Processing responses: 100%|██████████| 124/124 [09:36<00:00,  4.65s/item]


In [7]:
predicted_answers[:5]

['اسلامی کونسل کے رکن کی مدت کار تین سال ہوتی ہے۔',
 'پاکستان میں یوم آغاز سے قبل، بلوچستان اور خیبر پختونخوا کے قبائلی علاقے جات، امب، چترال، دیر اور سوات کی سابقہ ریاستیں کو قبائلی علاقے جات سمجھا جاتا تھا۔',
 'نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی کے بغیر کسی دوسری عدالت عالیہ میں تبادلہ نہیں کیا جا سکتا۔',
 'وفاقی حکومت یا صوبائی حکومت کے پاس ہے۔',
 'پارٹی کا سربراہ مذکورہ رکن کو اس بارے میں اظہار وجوہ کا موقع فراہم کرے گا کہ کیوں نہ اس کے خلاف مذکورہ اعلان کر دیا جائے۔']

In [8]:
df['prediction_text_mistral'] = predicted_answers

In [12]:
display(df.head())

,id,answers,prediction_text_mistral
0,0,اسلامی کونسل کا کوئی رکن تین سال کی مدت کے لیے...,اسلامی کونسل کے رکن کی مدت کار تین سال ہوتی ہے۔
1,1,پاکستان میں یوم آغاز سے قبل قبائلی علاقے جات م...,پاکستان میں یوم آغاز سے قبل، بلوچستان اور خیبر...
2,2,نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی ک...,نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی ک...
3,3,وفاقی حکومت یا، جیسی بھی صورت ہو، صوبائی حکومت...,وفاقی حکومت یا صوبائی حکومت کے پاس ہے۔
4,4,کسی رکن کو منحرف قرار دینے سے پہلے، پارٹی کے س...,پارٹی کا سربراہ مذکورہ رکن کو اس بارے میں اظہا...


In [13]:
df.to_csv('../../data/model_qa_responses/mistral-large-responses.csv', index=False)